<a href="https://colab.research.google.com/github/JASCRAJ/Sketch-literature/blob/main/12062024_svg_to_video.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install svgwrite cairosvg opencv-python


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.1/67.1 kB 793.4 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 3.9 MB/s eta 0:00:00


In [ ]:
import xml.etree.ElementTree as ET
import re

def parse_svg(svg_file):
    tree = ET.parse(svg_file)
    root = tree.getroot()
    namespaces = {'svg': 'http://www.w3.org/2000/svg'}

    viewBox = root.attrib.get('viewBox', '0 0 476 600')
    width, height = viewBox.split()[2:4]

    strokes = []
    for path in root.findall('.//svg:path', namespaces):
        d = path.attrib['d']
        fill = path.attrib.get('fill', 'none')
        cleaned_d = re.sub(r'\s+', ' ', d.strip())  # Remove extra whitespace
        strokes.append((cleaned_d, fill))

    return strokes, (width, height)


In [ ]:
import svgwrite
import cairosvg

def create_svg_with_strokes(strokes, size, output_file):
    width, height = size
    dwg = svgwrite.Drawing(output_file, profile='tiny', size=(f'{width}px', f'{height}px'), viewBox=f'0 0 {width} {height}')
    for stroke, fill in strokes:
        dwg.add(dwg.path(d=stroke, fill=fill, stroke='#213d46', stroke_width=2))
    dwg.save()

def svg_to_png(svg_file, png_file):
    cairosvg.svg2png(url=svg_file, write_to=png_file)


In [6]:
import cv2
import os

def generate_images_from_strokes(strokes, size, temp_dir):
    images = []
    for i in range(1, len(strokes) + 1):
        current_strokes = strokes[:i]
        svg_file = os.path.join(temp_dir, f'image_{i}.svg')
        png_file = os.path.join(temp_dir, f'image_{i}.png')

        create_svg_with_strokes(current_strokes, size, svg_file)
        svg_to_png(svg_file, png_file)

        images.append(png_file)

    return images

def create_video_from_images(image_files, output_video_file, fps=15):
    frame = cv2.imread(image_files[0])
    height, width, layers = frame.shape
    video = cv2.VideoWriter(output_video_file, cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

    for image_file in image_files:
        video.write(cv2.imread(image_file))

    cv2.destroyAllWindows()
    video.release()


In [7]:
import shutil

# List of SVG files to process
svg_files = ['/content/sketch0001_accumulated_clusters_2_38.svg', '/content/sketch0002_accumulated_clusters_2_40.svg']  # Add your SVG file paths here
output_videos = ['output_video1.mp4', 'output_video2.mp4']  # Corresponding output video file paths

for svg_file, output_video_file in zip(svg_files, output_videos):
    # Temporary directory for image files
    temp_dir = 'temp_images'

    # Ensure temp directory exists
    if os.path.exists(temp_dir):
        shutil.rmtree(temp_dir)
    os.makedirs(temp_dir)

    # Parse the SVG and get the strokes
    strokes, size = parse_svg(svg_file)

    # Generate images from strokes
    image_files = generate_images_from_strokes(strokes, size, temp_dir)

    # Create a video from the generated images
    create_video_from_images(image_files, output_video_file)

    # Clean up temp directory
    shutil.rmtree(temp_dir)

    print(f'Video saved to {output_video_file}')


Video saved to output_video1.mp4
Video saved to output_video2.mp4
